# Dataset



In [1]:
import os

train_infos = "/home/kdang/projects/spartDM/data/starmen/output_random_noacc/starmen_train.csv"
os.chdir("/home/kdang/projects/spartDM/")
print("Current working directory:", os.getcwd())

Current working directory: /home/kdang/projects/spartDM


In [2]:
from src.data import StarmenDataset
from torch.utils.data import DataLoader


starmen = StarmenDataset(train_infos)

train_loader = DataLoader(starmen, batch_size=2)

x, x_prev = next(iter(train_loader))

In [3]:
from src.sadm.vivit import ViViT
import numpy as np 


vivit = ViViT(
    image_size = (64, 64),
    patch_size = (16, 16),
    num_frames = 9,
    depth = 3,
    heads = 3,
    dim_head=8
)

In [45]:
from einops.layers.torch import Rearrange
from torch import nn
import torch
from einops import rearrange

patch_size = (16, 16) 
p1, p2 = patch_size
h, w = (4, 4)
patch_dim = int(np.prod(patch_size))


# c = vivit(x_prev)




temporal_layer = vivit.temporal_transformer.layers
len(temporal_layer)
vivit.temporal_dims


x_prev.shape
x = vivit.to_patch_embedding(x_prev)
b, t, n, d = x.shape
x += vivit.pos_embedding
x = vivit.dropout(x)
x = rearrange(x, "b t n d -> b (t n) d")
x = vivit.temporal_transformer(x)

print(f"After temporal transformer x.shape: {x.shape}")

# Perform space transformer

space_layer = vivit.space_transformer.layers
len(space_layer)

space_layer[3]

# vivit.space_dims



After temporal transformer x.shape: torch.Size([2, 144, 1])


ModuleList(
  (0): PreNorm(
    (norm): LayerNorm((144,), eps=1e-05, elementwise_affine=True)
    (fn): Attention(
      (to_qkv): Linear(in_features=144, out_features=72, bias=False)
      (to_out): Sequential(
        (0): Linear(in_features=24, out_features=144, bias=True)
        (1): Dropout(p=0.0, inplace=False)
      )
    )
  )
  (1): PreNorm(
    (norm): LayerNorm((144,), eps=1e-05, elementwise_affine=True)
    (fn): FeedForward(
      (net): Sequential(
        (0): Linear(in_features=144, out_features=144, bias=True)
        (1): GELU(approximate='none')
        (2): Dropout(p=0.0, inplace=False)
      )
    )
  )
  (2): Upsample(
    (net): Sequential(
      (0): Upsample(scale_factor=8.0, mode='nearest')
      (1): Rearrange('b t (u d) -> b (u t) d', d=144, u=8)
    )
  )
)